In [5]:
mini_json_path = "/workspace/data/annotation/label/mini.json"

In [6]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# 1. 环境适应性设置
plt.switch_backend('Agg') 
sns.set_context("paper", font_scale=1.5) # 设置为论文（paper）上下文模式
sns.set_style("ticks")

ann_map = {
    3: 1,
    4: 2,
	5: 3
}
def optimize_visual_report(json_file, output_dir="annotation_analysis"):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # 2. 数据解析
    with open(json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)

    all_data = []
    for entry in data:
        ann_id = f"Person_{ann_map.get(entry.get('annotator'), entry.get('annotator'))}"
        video_name = os.path.basename(entry.get('video')).split('=')[-1]
        
        for label_item in entry.get("videoLabels", []):
            direction = label_item.get("timelinelabels", ["unknown"])[0]
            for r in label_item.get("ranges", []):
                duration = r['end'] - r['start']
                all_data.append({
                    "Annotator": ann_id,
                    "Direction": direction,
                    "Duration": duration,
                    "Video": video_name
                })

    df = pd.DataFrame(all_data)

    # --- 绘图 A: 标注数量对比 (学术配色) ---
    plt.figure(figsize=(10, 6))
    ax1 = sns.countplot(data=df, x='Direction', hue='Annotator', palette='Set2')
    plt.title('Annotator Label Distribution (Count)', pad=20)
    plt.xlabel('Action Category')
    plt.ylabel('Number of Instances')
    sns.despine() # 去掉上方和右方的边框，更美观
    plt.savefig(f'{output_dir}/count_distribution.png', dpi=300, bbox_inches='tight')

    # --- 维度 A: 标注频率差异 (Stacked Bar Chart) ---
    plt.figure(figsize=(10, 6))
    pivot_df = df.groupby(['Annotator', 'Direction']).size().unstack(fill_value=0)
    pivot_df.plot(kind='bar', stacked=True, ax=plt.gca(), colormap='tab10')
    plt.title('Annotator Labeling Preference (Count)', fontsize=14)
    plt.ylabel('Total Number of Labels')
    plt.legend(title='Directions', bbox_to_anchor=(1.05, 1))
    plt.xticks(rotation=0)
    sns.despine()
    plt.tight_layout()
    plt.savefig(f"{output_dir}/annotator_preference.png", dpi=300)
    plt.close()

    # --- 生成统计汇总表 (Pivot Table) ---
    summary = df.groupby(['Annotator', 'Direction']).agg({
        'Duration': ['count', 'mean', 'std']
    }).reset_index()
    # 扁平化表头
    summary.columns = ['Annotator', 'Direction', 'Count', 'Avg_Duration', 'Std_Dev']
    summary.to_csv(f'{output_dir}/statistical_summary.csv', index=False)

    print(f"🚀 优化完成！图片和表格已保存在: {output_dir}/")

# 执行
optimize_visual_report(mini_json_path)

🚀 优化完成！图片和表格已保存在: annotation_analysis/


In [7]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import os

# Linux服务器环境支持
plt.switch_backend('Agg')

ann_map = {
    3: 1,
    4: 2,
	5: 3
}

def visualize_with_front_fill(json_file, output_dir="annotation_analysis/timeline_visuals"):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    with open(json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # 1. 颜色映射：增加 'front' 方向 (使用中性的灰色或淡蓝色)
    color_map = {
        'up': '#E74C3C', 'down': '#3498DB', 
        'left': '#2ECC71', 'right': '#F1C40F',
        'up-left': '#9B59B6', 'up-right': '#E67E22',
        'down-left': '#1ABC9C', 'down-right': '#34495E',
        'front': '#D5D8DC'  # 前方的颜色（淡灰色）
    }

    # 数据分组
    video_groups = {}
    for entry in data:
        raw_path = entry.get('video', 'unknown')
        video_name = raw_path.split('/')[-1] if '/' in raw_path else os.path.basename(raw_path)
        video_name = video_name.replace('.mp4', '')
        if video_name not in video_groups:
            video_groups[video_name] = []
        video_groups[video_name].append(entry)

    for video_name, entries in video_groups.items():
        plt.figure(figsize=(16, 8))
        
        # 获取标注者列表 (注意：你的数据中键名是 'id' 或 'annotation_id')
        annotators = sorted(list(set([str(ann_map[e.get('annotator')]) for e in entries])))
        ann_to_y = {ann_id: i for i, ann_id in enumerate(annotators)}

        for entry in entries:
            ann_id = str(ann_map[entry.get('annotator')])
            y_idx = ann_to_y[ann_id]
            
            # 提取该标注者所有的原始片段并排序
            all_segments = []
            for label_item in entry.get("videoLabels", []):
                label = label_item.get("timelinelabels", ["unknown"])[0]
                for r in label_item.get("ranges", []):
                    all_segments.append({'start': r['start'], 'end': r['end'], 'label': label})
            
            # 按时间排序
            all_segments.sort(key=lambda x: x['start'])

            # --- 核心逻辑：填充空白帧为 'front' ---
            filled_segments = []
            current_time = 0 
            
            # 如果视频起始不是从0开始，先填补开头的 front
            if all_segments and all_segments[0]['start'] > 0:
                filled_segments.append({'start': 0, 'end': all_segments[0]['start'], 'label': 'front'})

            for i, seg in enumerate(all_segments):
                # 检查当前片段与上一个片段之间是否有间隙
                if seg['start'] > current_time:
                    filled_segments.append({'start': current_time, 'end': seg['start'], 'label': 'front'})
                
                filled_segments.append(seg)
                current_time = max(current_time, seg['end'])

            # 绘制所有片段（包含填充的 front）
            for seg in filled_segments:
                color = color_map.get(seg['label'], '#95A5A6')
                plt.hlines(y=y_idx, xmin=seg['start'], xmax=seg['end'], 
                           color=color, linewidth=30, alpha=1.0) # alpha调高使颜色更实

        # 4. 图表修饰
        plt.yticks(range(len(annotators)), [f"Annotator {a}" for a in annotators], fontsize=12)
        plt.xlabel("Timeline (Frames/ms)", fontsize=12)
        plt.title(f"Comparison with: {video_name}", fontsize=16, pad=20)
        plt.grid(axis='x', linestyle=':', alpha=0.5)
        plt.ylim(-0.5, len(annotators) - 0.5)

        # 凡例
        legend_patches = [mpatches.Patch(color=color, label=label) for label, color in color_map.items()]
        plt.legend(handles=legend_patches, title="Directions", 
                   bbox_to_anchor=(1.02, 1), loc='upper left')

        save_path = os.path.join(output_dir, f"{video_name}.png")
        plt.tight_layout()
        plt.savefig(save_path, dpi=300)
        plt.close()
        print(f"Success: {video_name} visualization saved to {save_path}")

# 运行（请确保路径正确）
visualize_with_front_fill(mini_json_path)

Success: person_01_day_high_h265 visualization saved to annotation_analysis/timeline_visuals/person_01_day_high_h265.png
Success: person_01_day_low_h265 visualization saved to annotation_analysis/timeline_visuals/person_01_day_low_h265.png
Success: person_01_night_high_h265 visualization saved to annotation_analysis/timeline_visuals/person_01_night_high_h265.png
Success: person_01_night_low_h265 visualization saved to annotation_analysis/timeline_visuals/person_01_night_low_h265.png
Success: person_02_day_high_h265 visualization saved to annotation_analysis/timeline_visuals/person_02_day_high_h265.png
Success: person_02_day_low_h265 visualization saved to annotation_analysis/timeline_visuals/person_02_day_low_h265.png
Success: person_02_night_high_h265 visualization saved to annotation_analysis/timeline_visuals/person_02_night_high_h265.png
Success: person_02_night_low_h265 visualization saved to annotation_analysis/timeline_visuals/person_02_night_low_h265.png
Success: person_03_day_h